In [1]:
from riotwatcher import TftWatcher, ApiError
import importlib

In [7]:
#importlib.reload(secret)
import secret

RIOT_API_KEY = secret.RIOT_API_KEY

In [10]:
tft_watcher = TftWatcher(api_key=RIOT_API_KEY)
region = 'KR'

In [32]:
region_match = 'ASIA'

## 아이디들 수집

In [22]:
import pickle
summoner_names_file = 'data/summoner_names.txt'
summoner_puuid_file = 'data/summoner_puuid.txt'

In [11]:
challengers = tft_watcher.league.challenger(region)
grandmasters = tft_watcher.league.grandmaster(region)
masters = tft_watcher.league.master(region)
challengers = [summoner['summonerName'] for summoner in challengers['entries']]
grandmasters = [summoner['summonerName'] for summoner in grandmasters['entries']]
masters = [summoner['summonerName'] for summoner in masters['entries']]

In [12]:
top_tiers = challengers + grandmasters + masters

In [17]:
with open(summoner_names_file, 'wb') as out_file:
     pickle.dump(top_tiers, out_file)

In [ ]:
with open(summoner_names_file, 'rb') as in_file:
     top_tiers = pickle.load(in_file)

In [26]:
puuids = []
for name in top_tiers:
    try:
        response = tft_watcher.summoner.by_name(region, name)
        puuids.append(response['puuid'])
    except ApiError as err:
        print(err)

404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/ING%20%EC%9A%B0%EB%98%90
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%B2%B4%EC%8A%A4%ED%95%98%EB%9F%AC%EB%8F%8C%EC%95%84%EC%98%B4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/AFTV%20%EC%97%AD%EC%85%98%ED%99%98
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/ilsn
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EA%B0%90%20%EC%9E%83%EC%9D%80%20%EC%83%81%EC%A7%80
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%B8%8C%EB%9D%BC%EC%9E%90%EB%81%84%EB%84%A4%EB%81%BC%ED%86%B3
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%A0%95%EA%B8%80%EC%9C%84%EC%B9%98%EB%8B%A4%EC%95%8C%

404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/TFT%20%EC%9E%AC%ED%98%B8
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%9E%89%EC%9E%89%EC%96%B4%EC%96%B4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%8B%AC%EB%B9%9B%EC%97%90%20%EA%B7%B8%EB%A6%B0%20%EB%8A%91%EB%8C%80
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/MID%20GAP%20FUXK%20YOU
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/KeSPA%20FIFA
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/H1st%20%EB%82%98%EC%B7%A8%ED%96%88%EB%8B%A4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%9A%B4%EB%8F%84%EC%8B%A4%EB%A0%A5%EC%9D%B4%EB%8B%88%EA%B9%90
404 Client Error: Not Fo

404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/ALT%20F4%20BYE
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%8F%84%EA%B5%AC%EB%8A%94%20%EC%A1%B0%EC%9A%A9%ED%95%98%EB%8B%A4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%B0%95%ED%98%84%EC%9A%B1%20%EC%9E%A5%EA%B5%B0
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%90%9C%EC%9E%A5%EC%8C%88%EC%9F%9D
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%B0%94%EB%A1%A0%20%EB%82%A8%EC%B9%9C2
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/2%EC%83%81%EA%B7%9C
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/true%20Kapiyva
404 Client Error: Not Found for url: https://kr.api.riot

404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%88%AB%ED%83%80%EB%B2%85%EC%8A%A4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/shut%20the%20fk%20up
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%A1%B0%EC%9A%A9%ED%95%9C%20%ED%95%98%EB%8B%88
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%B2%AD%EC%9D%84%EB%93%A4%EC%96%B4%EB%9D%BC
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%97%84%EB%B3%91%EB%AF%BC
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%B9%A0%ED%9D%91%EB%AC%BC%EA%B1%B8%EB%8B%A4%EC%9D%B4%EC%95%A0%EB%82%98
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%88%88%EC%9B%83%EC%9D%8C%EB%A7%A4%EB%A0%A5%

404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EC%B0%B8%20%EC%89%BD%EB%84%A4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%82%A0%20%EB%B9%84%EC%9B%83%EB%8B%A4
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/l%EB%98%A5%EB%AC%BC%EB%A6%AC%EC%97%90l
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EA%B2%A8%20%EC%B2%A8
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%EB%B9%84%EB%8A%94%EB%84%88%EB%AC%B4%EC%B0%A8%EA%B0%80%EC%9B%8C
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/Grindingazz
404 Client Error: Not Found for url: https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/%ED%95%98%EC%95%84%EC%96%80%EC%83%88
404 Client Error: Not Found for url: https://kr.api.rio

In [27]:
with open(summoner_puuid_file, 'wb') as out_file:
     pickle.dump(puuids, out_file)

## 매치데이터 수집

In [90]:
from datetime import datetime

In [91]:
temp_date_string = '2021-04-27 12:00:00'
temp_start_date = datetime.strptime(temp_date_string, '%Y-%m-%d %H:%M:%S')
temp_start_timestamp = int(datetime.timestamp(temp_start_date) * 1000)

default_date_string = '2018-01-01 00:00:00'
default_start_date = datetime.strptime(default_date_string, '%Y-%m-%d %H:%M:%S')
default_start_timestamp = int(datetime.timestamp(default_start_date) * 1000)

season5_date_string = '2021-04-28 11:00:00'
season5_start_date = datetime.strptime(season5_date_string, '%Y-%m-%d %H:%M:%S')
season5_start_timestamp = int(datetime.timestamp(season5_start_date) * 1000)
# datetime.fromtimestamp(int(mates[0]['info']['game_datetime']) / 1000)

In [20]:
puuids = None
with open(summoner_puuid_file, 'rb') as in_file:
     puuids = pickle.load(in_file)

In [80]:
def GetMatchIdsByList(list_puuids: list, count: int = 20):
    local_match_ids = []
    for puuid in list_puuids:
        try:
            response = tft_watcher.match.by_puuid(region_match, str(puuid), 20)
            local_match_ids.extend(response)
        except ApiError as err:
            print(err)
    return local_match_ids

In [93]:
def GetMatchesByList(list_match_ids: list, limit_start_timestamp: int = default_start_timestamp):
    local_matches = []
    for match_id in list_match_ids:
        try:
            response = tft_watcher.match.by_id(region_match, match_id)
            if (response['info']['game_datetime'] < limit_start_timestamp):
                break
            local_matches.append(response)
        except ApiError as err:
            print(err)
    return local_matches

In [94]:
match_ids = GetMatchIdsByList(puuids[0:5], 20)
match_ids = sorted(list(set(match_ids)), reverse=True)

In [102]:
matches = GetMatchesByList(match_ids[0:5], temp_start_timestamp)

## 수집데이터 가공

In [99]:
import pandas as pd
import numpy as np

In [111]:
df = pd.DataFrame(matches[0]['info']['participants'])

In [123]:
pd.DataFrame.from_dict(matches[0]['info']['participants'][0]['traits'])

,name,num_units,style,tier_current,tier_total
0,Cultist,1,0,0,3
1,Fortune,1,0,0,2
2,Set4_Adept,1,0,0,3
3,Set4_Blacksmith,1,3,1,1
4,Set4_Brawler,1,0,0,4
5,Set4_Dragonsoul,1,0,0,3
6,Set4_Elderwood,1,0,0,3
7,Set4_Exile,1,1,1,2
8,Set4_Fabled,3,3,1,1
9,Set4_Mystic,2,1,1,3


In [112]:
df

,companion,gold_left,last_round,level,placement,players_eliminated,puuid,time_eliminated,total_damage_to_players,traits,units
0,{'content_ID': 'db9cbd4d-06a0-4361-9200-e6fa86...,0,38,9,3,1,68px1xvgwamVhypeoOloVD5skZcGz99YkT_RNAQ2C4P_oI...,2193.697998,117,"[{'name': 'Cultist', 'num_units': 1, 'style': ...","[{'character_id': 'TFT4_Braum', 'items': [], '..."
1,{'content_ID': '8f032d75-83f7-445c-b2dd-751dc2...,38,31,8,7,0,5BQ1ICCPnwHjqwsF5NJTd_Amb7zGsJ0sYXn2kIX9aUC21q...,1783.637939,85,"[{'name': 'Cultist', 'num_units': 2, 'style': ...","[{'character_id': 'TFT4_Brand', 'items': [], '..."
2,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,0,34,8,4,0,rgVewqGg_q5KPE4_LXus6RQHNdilF_-3oa2k13XoxJKaDE...,1979.460205,71,"[{'name': 'Cultist', 'num_units': 2, 'style': ...","[{'character_id': 'TFT4_Tristana', 'items': []..."
3,{'content_ID': '5897ad9f-4665-4372-8f3e-6c878a...,0,34,8,5,0,EnKE4hH3fSTTdXao6LFhLQ3aBey6j_n2-kxozwDta_eZer...,1976.616089,82,"[{'name': 'Cultist', 'num_units': 9, 'style': ...","[{'character_id': 'TFT4_Elise', 'items': [], '..."
4,{'content_ID': 'b1f08eb3-c601-4ca1-a6c7-15e892...,35,40,8,1,4,OmUhykxCkne00zurXn9eOCzuGmAlRVJdXqCtdomH_etjld...,2308.046387,158,"[{'name': 'Divine', 'num_units': 6, 'style': 3...","[{'character_id': 'TFT4_Jax', 'items': [], 'na..."
5,{'content_ID': 'db9cbd4d-06a0-4361-9200-e6fa86...,24,26,7,8,0,-iS0SGU2PhdYolJzLGIiGCwGnZXtpJYUkTE1f4GijfBx6E...,1457.922729,60,"[{'name': 'Cultist', 'num_units': 1, 'style': ...","[{'character_id': 'TFT4_TahmKench', 'items': [..."
6,{'content_ID': '91f2e228-4e36-4dad-9a97-36036e...,1,33,8,6,0,VyKTsVPGL9n1XYrH6TwcSQa4C7HC8vAqZrwlu0oMuNkGT4...,1898.508179,99,"[{'name': 'Cultist', 'num_units': 3, 'style': ...","[{'character_id': 'TFT4_Pyke', 'items': [], 'n..."
7,{'content_ID': 'af36c3f4-6377-47ca-abe9-943f92...,0,40,9,2,1,56QqlwB3zIC4caqyB4mCfMqxnANBqXtP3vpNy02RkRono3...,2303.632080,124,"[{'name': 'Cultist', 'num_units': 2, 'style': ...","[{'character_id': 'TFT4_Nautilus', 'chosen': '..."
